In [1]:
from sklearn.feature_selection import mutual_info_classif           # Information Gain for classification task
from sklearn.feature_selection import chi2                          # chi-square feature selection method
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE                           # Recurssive feature elimination algorithm
from sklearn.feature_selection import SequentialFeatureSelector     # Forward feature selectioin method
from sklearn.linear_model import LassoCV , RidgeCV                  # lasso and ridge cross validation algorithms for feature selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel               # to select features according to prefit LassoCV and RidgeCV
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np
from sklearn.preprocessing import StandardScaler


### Dataset Loading ###

In [2]:
PATH = './data/ld5/train.csv'
Target_Variable = 'Status'

frame = pd.read_csv(PATH)

In [3]:
frame.head()

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,...,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,65087372,10000,32236,12329.36286,59,BAT2522922,11.135007,B,C4,MORTGAGE,...,2.498291,0.793724,0,INDIVIDUAL,49,0,31,311301,6619,0
1,1450153,3609,11940,12191.99692,59,BAT1586599,12.237563,C,D3,RENT,...,2.377215,0.974821,0,INDIVIDUAL,109,0,53,182610,20885,0
2,1969101,28276,9311,21603.22455,59,BAT2136391,12.545884,F,D4,MORTGAGE,...,4.316277,1.020075,0,INDIVIDUAL,66,0,34,89801,26155,0
3,6651430,11170,6954,17877.15585,59,BAT2428731,16.731201,C,C3,MORTGAGE,...,0.107020,0.749971,0,INDIVIDUAL,39,0,40,9189,60214,0
4,14354669,16890,13226,13539.92667,59,BAT5341619,15.008300,C,D4,MORTGAGE,...,1294.818751,0.368953,0,INDIVIDUAL,18,0,430,126029,22579,0


In [4]:
frame.isnull().sum()

ID                              0
Loan Amount                     0
Funded Amount                   0
Funded Amount Investor          0
Term                            0
Batch Enrolled                  0
Interest Rate                   0
Grade                           0
Sub Grade                       0
Employment Duration             0
Home Ownership                  0
Verification Status             0
Payment Plan                    0
Loan Title                      0
Debit to Income                 0
Delinquency - two years         0
Inquires - six months           0
Open Account                    0
Public Record                   0
Revolving Balance               0
Revolving Utilities             0
Total Accounts                  0
Initial List Status             0
Total Received Interest         0
Total Received Late Fee         0
Recoveries                      0
Collection Recovery Fee         0
Collection 12 months Medical    0
Application Type                0
Last week Pay 

In [5]:
for i in frame.columns[frame.isna().sum() > 0]:
    frame[i] = frame[i].interpolate('linear')

In [6]:
frame.isnull().sum()

ID                              0
Loan Amount                     0
Funded Amount                   0
Funded Amount Investor          0
Term                            0
Batch Enrolled                  0
Interest Rate                   0
Grade                           0
Sub Grade                       0
Employment Duration             0
Home Ownership                  0
Verification Status             0
Payment Plan                    0
Loan Title                      0
Debit to Income                 0
Delinquency - two years         0
Inquires - six months           0
Open Account                    0
Public Record                   0
Revolving Balance               0
Revolving Utilities             0
Total Accounts                  0
Initial List Status             0
Total Received Interest         0
Total Received Late Fee         0
Recoveries                      0
Collection Recovery Fee         0
Collection 12 months Medical    0
Application Type                0
Last week Pay 

In [7]:

cat_vars = ['Batch Enrolled','Grade','Sub Grade','Employment Duration','Verification Status','Payment Plan',,'Loan Title'.'Initial List Status','Application Type']

for i in cat_vars:
    le = LabelEncoder()
    frame[i] = le.fit_transform(frame[i])

SyntaxError: invalid syntax (3462716748.py, line 1)

In [ ]:
[frame[i].iloc[0:10] for i in frame.columns]

[0    65087372
 1     1450153
 2     1969101
 3     6651430
 4    14354669
 5    50509046
 6    32737431
 7    63151650
 8     4279662
 9     4431034
 Name: ID, dtype: int64,
 0    10000
 1     3609
 2    28276
 3    11170
 4    16890
 5    34631
 6    30844
 7    20744
 8     9299
 9    19232
 Name: Loan Amount, dtype: int64,
 0    32236
 1    11940
 2     9311
 3     6954
 4    13226
 5    30203
 6    19773
 7    10609
 8    11238
 9     8962
 Name: Funded Amount, dtype: int64,
 0    12329.362860
 1    12191.996920
 2    21603.224550
 3    17877.155850
 4    13539.926670
 5     8635.931613
 6    15777.511830
 7     7645.014802
 8    13429.456610
 9     7004.097481
 Name: Funded Amount Investor, dtype: float64,
 0    59
 1    59
 2    59
 3    59
 4    59
 5    36
 6    59
 7    58
 8    59
 9    58
 Name: Term, dtype: int64,
 0    BAT2522922
 1    BAT1586599
 2    BAT2136391
 3    BAT2428731
 4    BAT5341619
 5    BAT4694572
 6    BAT4808022
 7    BAT2558388
 8    BAT5341619
 9    BA

In [ ]:
frame = frame.dropna()
frame

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
2,24892,2019,0,2,1,0,0,0,0,1,...,3,834,0,1,1,80.019685,3,1,0,356
3,24893,2019,0,2,0,0,3,0,0,1,...,3,587,0,2,0,69.376900,0,1,0,308
4,24894,2019,0,1,1,0,0,0,0,1,...,1,602,1,0,0,91.886544,0,1,0,273
5,24895,2019,0,1,1,0,0,0,0,1,...,3,864,1,1,0,70.089286,0,1,0,285
6,24896,2019,0,1,1,0,2,0,0,1,...,3,860,1,3,1,79.109589,0,1,0,334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148665,173555,2019,0,3,0,0,2,0,0,1,...,0,659,1,3,1,71.792763,3,1,0,380
148666,173556,2019,0,2,0,0,0,0,0,1,...,0,569,0,0,0,74.428934,3,1,0,37
148667,173557,2019,0,2,0,0,3,0,0,1,...,0,702,1,2,0,61.332418,0,1,0,390
148668,173558,2019,0,0,0,0,3,0,0,1,...,3,737,1,3,1,70.683453,0,1,0,160


In [ ]:

X = frame.drop([Target_Variable],axis=1)
Y = frame[Target_Variable]


In [ ]:
# Identify the minority class label
unique, counts = np.unique(Y, return_counts=True)
minority_class = unique[np.argmin(counts)]

# Get the data belonging to the minority class
minority_X = X[Y == minority_class]
minority_y = Y[Y == minority_class]

# Get the majority class data (optional, for combining later)
majority_X = X[Y != minority_class]
majority_y = Y[Y != minority_class]

# Upsample the minority class to match the size of the majority class
n_samples = len(majority_X)  # Number of samples in the majority class
upsampled_minority = resample(minority_X, replace=True, n_samples=n_samples, random_state=42)
upsampled_minority_y = np.full(len(upsampled_minority), minority_class)  # Assign class labels

# Combine the upsampled data with the majority class (optional)
X_combined = np.concatenate((upsampled_minority, majority_X))
y_combined = np.concatenate((upsampled_minority_y, majority_y))


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)

X = pd.DataFrame(X_scaled,columns=X.columns)
Y = y_combined





print(X.shape)
print(Y.shape)

(224062, 33)
(224062,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

### Chi-Square Feature Selection ###

In [ ]:
selector = VarianceThreshold(threshold=1)  # Remove features with variance < 0.01
selector.fit(X)
print(selector.get_support())

[False False  True False False False  True False False False  True  True
 False  True False False False False False False  True False  True  True
  True False False  True False  True False False  True]


In [ ]:
selected_features = X.columns[selector.get_support()] 
print("\nSelected Features:") 
print(selected_features)


Selected Features:
Index(['loan_limit', 'loan_purpose', 'loan_amount', 'rate_of_interest',
       'Upfront_charges', 'occupancy_type', 'total_units', 'income',
       'credit_type', 'age', 'LTV', 'dtir1'],
      dtype='object')


In [ ]:
clf = LogisticRegression()
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.5911064226883597

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.9678662717572118

### Mutual Info ###

In [ ]:
fs = mutual_info_classif(X,Y)

In [ ]:
selected_features = X.columns[fs > 0.05] 
print("\nSelected Features:") 
print(selected_features)


Selected Features:
Index(['ID', 'rate_of_interest', 'Interest_rate_spread', 'Upfront_charges',
       'property_value', 'credit_type', 'LTV', 'dtir1'],
      dtype='object')


In [ ]:
clf = LogisticRegression(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.6117580232888384

In [ ]:
clf = RandomForestClassifier(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.9695432845106233

### Forward Selection Method ###

In [ ]:
clf = LogisticRegression(n_jobs=16)
sfs = SequentialFeatureSelector(clf,n_features_to_select=5)
sfs.fit(X, Y)
sfs.get_support()

array([False,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False, False, False, False, False, False])

In [ ]:
selected_features = X.columns[sfs.get_support()] 
print("\nSelected Features:") 
print(selected_features)


Selected Features:
Index(['year', 'Credit_Worthiness', 'Neg_ammortization', 'construction_type',
       'credit_type'],
      dtype='object')


In [ ]:
clf = LogisticRegression(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.654670615761215

In [ ]:
clf = RandomForestClassifier(n_jobs=16)
sfs = SequentialFeatureSelector(clf,n_features_to_select=5,n_jobs=16)
sfs.fit(X, Y)
sfs.get_support()

array([False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True, False, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False])

In [ ]:
selected_features = X.columns[sfs.get_support()] 
print("\nSelected Features:") 
print(selected_features)


Selected Features:
Index(['rate_of_interest', 'Interest_rate_spread', 'Upfront_charges',
       'credit_type', 'Credit_Score'],
      dtype='object')


In [ ]:
clf = RandomForestClassifier(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.9796053610310923

### Recurssive Feature Elimination ###

In [ ]:
estimator = LogisticRegression(n_jobs=16)
selector = RFE(estimator, n_features_to_select=5, step=0.2)
selector.fit(X, Y)
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False,  True,
       False, False, False, False, False, False,  True, False,  True,
       False,  True, False, False, False, False])

In [ ]:
selected_features = X.columns[selector.support_] 
print("\nSelected Features:") 
print(selected_features)


Selected Features:
Index(['Neg_ammortization', 'lump_sum_payment', 'credit_type',
       'co-applicant_credit_type', 'submission_of_application'],
      dtype='object')


In [ ]:
clf = LogisticRegression(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.6325448668533019

In [ ]:
estimator = RandomForestClassifier(n_jobs=16)
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X, Y)
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False,  True, False, False, False])

In [ ]:
selected_features = X.columns[selector.support_] 
print("\nSelected Features:") 
print(selected_features)


Selected Features:
Index(['rate_of_interest', 'Interest_rate_spread', 'Upfront_charges',
       'credit_type', 'LTV'],
      dtype='object')


In [ ]:
clf = RandomForestClassifier(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.9779148239812824

### Lasso Regx Feature Selection ###

In [ ]:
clf = LassoCV()
clf.fit(X, Y)
clf.coef_

array([-1.04068289e-03,  0.00000000e+00,  2.02212252e-02,  1.28218298e-02,
       -6.84943926e-03, -1.31266053e-02, -1.41520539e-02,  1.26125866e-02,
       -8.63711134e-03, -4.90824613e-02,  1.99606905e-02,  5.00763849e-04,
       -2.82477567e-02, -8.74756060e-03, -5.80594345e-06, -7.39015648e-02,
       -1.37805741e-02, -8.39940100e-02, -1.00759938e-02, -8.51224233e-03,
       -1.92690237e-02,  4.56912338e-05,  1.73005310e-02, -3.40582130e-02,
        5.16923690e-02,  1.16248600e-03,  8.10028683e-02,  1.38019690e-02,
        6.87152966e-02,  1.14180895e-02,  8.75516858e-03, -4.78062998e-04,
        2.29424544e-02])

In [ ]:
selected_features = X.columns[clf.coef_ != 0]
print("\nSelected Features:") 
print(selected_features)


Selected Features:
Index(['ID', 'loan_limit', 'Gender', 'approv_in_adv', 'loan_type',
       'loan_purpose', 'Credit_Worthiness', 'open_credit',
       'business_or_commercial', 'loan_amount', 'rate_of_interest',
       'Interest_rate_spread', 'Upfront_charges', 'term', 'Neg_ammortization',
       'interest_only', 'lump_sum_payment', 'property_value',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'income', 'credit_type', 'Credit_Score', 'co-applicant_credit_type',
       'age', 'submission_of_application', 'LTV', 'Region', 'Security_Type',
       'dtir1'],
      dtype='object')


In [ ]:
clf = LogisticRegression(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.6643945848717221

In [ ]:
clf = RandomForestClassifier(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.9737087678013552

### Ridge Regression For Feature Selection ###

In [ ]:
clf = RidgeCV()
clf.fit(X, Y)
clf.coef_

array([-0.00112659,  0.        ,  0.02028799,  0.0128845 , -0.00692476,
       -0.01334941, -0.01426027,  0.01268939, -0.00873509, -0.04932564,
        0.02033588,  0.00077099, -0.02861792, -0.00884032, -0.00019734,
       -0.07390715, -0.01390749, -0.08407266, -0.0103361 , -0.00303921,
       -0.01936973,  0.00303921,  0.01737062, -0.03422534,  0.0517613 ,
        0.00124363,  0.0810806 ,  0.01393533,  0.06885138,  0.0114624 ,
        0.00878821, -0.00303921,  0.02298394])

In [ ]:
selected_features = X.columns[clf.coef_ > 0.01]
print("\nSelected Features:") 
print(selected_features)


Selected Features:
Index(['loan_limit', 'Gender', 'Credit_Worthiness', 'loan_amount',
       'total_units', 'credit_type', 'co-applicant_credit_type', 'age',
       'submission_of_application', 'LTV', 'dtir1'],
      dtype='object')


In [ ]:
clf = LogisticRegression(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.6118526933636278

In [ ]:
clf = RandomForestClassifier(n_jobs=16)
clf.fit(X_train[selected_features],y_train)
score = clf.score(X_test[selected_features],y_test)
score

0.9313236228885192